In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler
# turn off autoreload so that we can use the old model 
# when editing the current project

from coeditor.common import *
import os
from coeditor.model import RetrievalEditorModel, AttentionMode, DecodingArgs
from coeditor.service import EditPredictionService, ChangeDetector, BatchArgs
from coeditor.c3problem import C3GeneratorCache

os.chdir(proj_root())

In [ ]:
# model.push_to_hub(f"MrVPlusOne/{model_name}")

In [ ]:
# target_dir = proj_root() / "../SPOT-copy/"
target_dir = proj_root()
model_name = "coeditor-xl-c3-dropout-v1.4"
model_path = get_model_dir(True) / model_name
model = RetrievalEditorModel.load(model_path)
model.to("cuda:0")
detector = ChangeDetector(target_dir)
service = EditPredictionService(
    detector,
    model,
    dec_args = DecodingArgs(do_sample=False, num_beams=6, length_penalty=0.0)
    # dec_args = DecodingArgs(do_sample=True, top_p=0.95, marginalize_samples=50)
)


In [ ]:
# service.dec_args = DecodingArgs(do_sample=True, top_p=0.9, marginalize_samples=25)
# service.dec_args = DecodingArgs(do_sample=False, num_beams=8, length_penalty=0.0)
service.tlogger.clear()
response = service.suggest_edit(Path("scripts/train_model.py"), 91)
print(response)
service.tlogger.as_dataframe()

In [ ]:
from pprint import pprint
pprint(service.detector.analyzer.error_counts)